# Monte Carlo Linear Programming with Python

***Piotr Skalski - 01.05.2018***

### 1. Imports

In [2]:
import random
import datetime
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

### 2. Model settings

In [49]:
# The range of values from which random coordinates will be generated
MAX_POINT_COORDINATE_VALUE = 20
# The number of random points generated as part of a single iteration
NUMBER_OF_RANDOM_POINTS_PER_SWEEP = 100000
# The maximum allowable difference in results obtained in subsequent iterations
EPS = 1

### 3. LinearProgrammingModel class definition

In [50]:
class LinearProgrammingModel():
    
    def __init__(self, points_per_sweep=100000, max_cord_value=100000, max_eps=10, verbose=True, no_improvement_max_sweep = 3):
        self.points_per_sweep = points_per_sweep
        self.max_cord_value = max_cord_value
        self.max_eps = max_eps
        self.verbose = verbose
        self.scores_after_iterations = []
        self.point_after_iterations = []
        self.no_improvement_max_sweep = no_improvement_max_sweep
        self.no_improvement_count = 0
        
        self.dimensions = None
        self.equation = None
        self.optimization = None
        self.conditions = None
        
        self.supported_operations = {
            "log" : "np.log",
            "sin" : "np.sin",
            "cos" : "np.cos",
            "tan" : "np.tan",
            "sqrt" : "np.sqrt",
            "absolute" : "np.absolute"
        }
        
    def fit(self, dimensions, equation, optimization, conditions):
        self.dimensions = dimensions
        self.equation = equation
        self.optimization = optimization
        self.conditions = conditions
        self.point_after_iterations = [[] for i in range(self.dimensions)]
        
        self.model_preprocessing()
        
        r = self.max_cord_value/2
        center = LinearProgrammingModel.generate_start_point(self.dimensions, int(r))
        prev_score = 100000000

        while True:
            point, score = self.single_sweep(center, r)
            
#             self.scores_after_iterations.append(score)
#             for i in range(self.dimensions):
#                 self.point_after_iterations[i].append(point["x" + str(i+1)])
            
            if self.verbose:
                print("Score: " + str(score))
                print("R: " + str(r))
                print("Point: " + str(point))
                print("-" * 80)
   
            if abs(prev_score - score) <= self.max_eps:
                self.no_improvement_count += 1
                if self.no_improvement_max_sweep == self.no_improvement_count:
                    return point, score
                else:
                    prev_score = score
                    center = point
            else:
                self.no_improvement_count = 0
                prev_score = score
                center = point
                r = int(r/2)
        
    def single_sweep(self, center, r):
        best_point = None
        best_score = None

        if self.validate_single_point(center):
            best_point = center
            equation = LinearProgrammingModel.replace_all(self.equation, center)
            best_score = eval(equation)

        for i in range(self.points_per_sweep):
            random_point = LinearProgrammingModel.generate_point_in_vicinity(center, r)

            if self.validate_single_point(random_point):
                equation = LinearProgrammingModel.replace_all(self.equation, random_point)
                value = eval(equation)
                if self.optimization is "max" and (best_score == None or best_score < value):
                    best_point = random_point
                    best_score = value
                elif self.optimization is "min" and (best_score == None or best_score > value):
                    best_point = random_point
                    best_score = value

        return best_point, best_score
        
    def model_preprocessing(self):
        self.conditions = [LinearProgrammingModel.replace_all(condition, self.supported_operations) for condition in self.conditions]
        self.equation = LinearProgrammingModel.replace_all(self.equation, self.supported_operations)

    def validate_single_point(self, point_position):
        return all([LinearProgrammingModel.evaluate_condition(condition, point_position) for condition in self.conditions])
    
    @staticmethod
    def generate_start_point(dimensions, value):
        return {"x"+str(i): str(value) for i in range(1, dimensions+1)}
    
    @staticmethod
    def replace_all(text, dic):
        for i, j in dic.items():
            text = text.replace(i, j)
        return text
    
    @staticmethod
    def evaluate_condition(condition, point_position):
        condition = LinearProgrammingModel.replace_all(condition, point_position)
        return eval(condition)
        
    @staticmethod
    def generate_point_in_vicinity(point, r):
        return {key: str(value + LinearProgrammingModel.random_number_in_vicinity(r)) for key, value in point.items()}
    
    @staticmethod
    def random_number_in_vicinity(r):
        return random.randint(0,2*r) - r

### 4. Problem definition

In [51]:
dimensions = 3
equation = "5*x1 + 4*x2 + 3*x3"
optimization = "max"
conditions = [
    "2*x1 + 3*x2 + x3 <= 5",
    "4*x1 + x2 + 2*x3 <= 11",
    "3*x1 + 4*x2 + 2*x3 <= 8",
    "x1 >= 0",
    "x2 >= 0",
    "x3 >= 0",
]

### 5. Solve problem

In [52]:
lp_model = LinearProgrammingModel(points_per_sweep=NUMBER_OF_RANDOM_POINTS_PER_SWEEP,
                                 max_cord_value=MAX_POINT_COORDINATE_VALUE,
                                 max_eps=EPS,
                                 verbose=True)

lp_model.fit(dimensions=dimensions, equation=equation, optimization=optimization, conditions=conditions)

Score: None
R: 10.0
Point: None
--------------------------------------------------------------------------------


TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'

### 6. Visualization of the solution

In [ ]:
def make_plot(x_values, y_values, x_name, y_name, plot_name, trace_color_marker, trace_color_line):

    trace = go.Scatter(
        x = x_values,
        y = y_values,
        name=y_name,
        mode = 'lines+markers',
        marker=dict(
            color=trace_color_marker,
            line=dict(
                color=trace_color_line,
                width=2,
            ),
            symbol="hexagon-dot",
            size=8
        ),
    )

    layout = go.Layout(
        barmode='stack',
        title=plot_name,
        titlefont=dict(size=25),
        width=750,
        height=500,
        paper_bgcolor='rgb(244, 238, 225)',
        plot_bgcolor='rgb(244, 238, 225)',
        yaxis = dict(
            title=y_name,
            anchor = 'x'
        ),
        xaxis = dict(title=x_name)
    )

    fig = go.Figure(data=[trace], layout=layout)
    py.iplot(fig)

In [ ]:
labels = list(range(1, len(lp_model.scores_after_iterations) + 1))

make_plot(labels, 
          lp_model.scores_after_iterations, 
          'Iterations', 
          'Function value per iteration', 
          'Convergence of optimized function', 
          'rgba(55, 128, 191, 0.7)', 
          'rgba(55, 128, 191, 1.0)')

make_plot(labels, 
          lp_model.point_after_iterations[0], 
          'Iterations', 
          'Value of x1 per iteration', 
          'Convergence of x1 coordinate', 
          'rgba(219, 64, 82, 0.7)', 
          'rgba(219, 64, 82, 1.0)')

make_plot(labels, 
          lp_model.point_after_iterations[1], 
          'Iterations', 
          'Value of x2 per iteration', 
          'Convergence of x2 coordinate', 
          'rgba(0, 168, 107, 0.7)', 
          'rgba(0, 168, 107, 1.0)')

make_plot(labels, 
          lp_model.point_after_iterations[2], 
          'Iterations', 
          'Value of x3 per iteration', 
          'Convergence of x3 coordinate', 
          'rgba(250, 92, 0, 0.7)', 
          'rgba(250, 92, 0, 1.0)')